In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
#from midas import m
#from datetime import datetime, date
#import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/CRC_methylation_caller_result_check_01202025"

## Data preocessing

In [2]:
# Readin the Infinity batch
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20241114_AstraZeneca_SOW202_Batch1_80Samples_InfinityReport.csv", header = 1) 

# Jill's batches
# df_ori = pd.read_csv("/ghsfa/projects/pharma/customers/Pfizer/Infinity/PFE_SOW51core_Infinity/PFE_SOW51core_Infinity_03/data/reports/20250121_Pfizer_SOW51core_Batch3_18Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250117_Pfizer_SOW46Amendment1TP1_Batch9_1Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250117_Pfizer_SOW46Amendment1TP1_Batch10_3Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250120_Pfizer_SOW46Amendment1TP3_Batch9_20Samples_G360Report.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250120_Pfizer_SOW46Amendment1TP3_Batch10_8Samples_G360Report.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250117_Pfizer_SOW46Amendment1TP5_Batch6_11Samples_InfinityReport.csv", header = 1) 
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20250120_Pfizer_SOW46Amendment1TP5_Batch7_33Samples_InfinityReport.csv", header = 1) 


# Mingyang's batches
# df_ori = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/data/infinity_report_warehouse/20241220_GuardantHealthIST_IST01_Batch1_405Samples_InfinityReport.csv", header = 1) 

# Haiyang's batch
df_ori = pd.read_csv("/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/XTX_TFv2_update_Batch1to40_01162025/df_xtx_Sirius161.csv", header = 0) 


print("Total sample size of the Infinity batch is " + str(df_ori.shape[0]) + "\n") 
print("Total unique sample size of the Infinity batch is " + str(df_ori.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of the Infinity batch is " + str(df_ori.Patient_ID.nunique()) + "\n")

df_ori.head()

Total sample size of the Infinity batch is 42

Total unique sample size of the Infinity batch is 42

Total unique patient sample size of the Infinity batch is 32



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,XTX101-0102-001,NaN,A1226443,A122644301,140-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.003628,58.33,NaN,NaN,11/30/24,11/27/24,12/13/24
1,XTX101-0102-001,NaN,A1203278,B00831229,121-009,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.146274,20.15,4.5,NaN,11/7/24,11/6/24,12/2/24
2,XTX101-0102-001,NaN,A1204902,B00832399,141-005,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.009578,195.64,4.5,NaN,11/8/24,11/7/24,11/25/24
3,XTX101-0102-001,NaN,A1204331,B00831858,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.304769,21.90,4.0,NaN,11/8/24,11/7/24,11/25/24
4,XTX101-0102-001,NaN,A1238988,A123898801,104-023,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.430250,26.06,NaN,NaN,12/12/24,12/11/24,1/9/25


In [3]:
# Remove duplication
df_dedup = df_ori.drop_duplicates(subset = "GHSampleID")
df_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date
0,XTX101-0102-001,NaN,A1226443,A122644301,140-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.003628,58.33,NaN,NaN,11/30/24,11/27/24,12/13/24
1,XTX101-0102-001,NaN,A1203278,B00831229,121-009,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.146274,20.15,4.5,NaN,11/7/24,11/6/24,12/2/24
2,XTX101-0102-001,NaN,A1204902,B00832399,141-005,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.009578,195.64,4.5,NaN,11/8/24,11/7/24,11/25/24
3,XTX101-0102-001,NaN,A1204331,B00831858,141-006,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.304769,21.90,4.0,NaN,11/8/24,11/7/24,11/25/24
4,XTX101-0102-001,NaN,A1238988,A123898801,104-023,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.430250,26.06,NaN,NaN,12/12/24,12/11/24,1/9/25


In [4]:
# Pull "runid" from ghdb
df_1 = df_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_1 = df_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_1 = df_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_1 = df_1.drop(columns='run_date')

print("Total sample size of the Infinity batch after merging runid is " + str(df_1.shape[0])) 
df_1.head()


Total sample size of the Infinity batch after merging runid is 42


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid
21,XTX101-0102-001,NaN,A1210253,A121025301,104-017,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.346847,42.92,4.5,NaN,11/14/24,11/13/24,11/25/24,241121_A01810_0520_BHT73GDSXC
13,XTX101-0102-001,NaN,A1226429,A122642901,146-004,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Detected,Detected,0.119741,37.01,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC
18,XTX101-0102-001,NaN,A1226435,A122643501,146-003,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.057710,77.38,NaN,NaN,11/30/24,11/29/24,12/13/24,241205_A00277_1180_AHTGM7DSXC
0,XTX101-0102-001,NaN,A1226443,A122644301,140-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.003628,58.33,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC
28,XTX101-0102-001,NaN,A1226445,A122644501,140-005,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.012115,49.20,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC


In [ ]:
# # Optional step!!!
# # If you only have one sample in the batch, run this cell alternatively instead of from above one

# # Pull "runid" from ghdb
# df_1 = df_dedup.copy()

# # Define the IDs you want to query for
# ids = tuple(df_1['GHSampleID'].unique())

# # If there's only one ID, format it without a comma
# if len(ids) == 1:
#     ids = f"('{ids[0]}')"
# else:
#     ids = str(ids)

# # Generate the SQL query string
# query = f"SELECT * FROM gh_sample WHERE run_sample_id IN {ids}"

# # Get the dataframe with "runid"
# sample_df = functions.run_sql_query(query)
# sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# # Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# # Extract the date from runid and convert to datetime format for accurate comparison
# sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
# df_1 = df_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
# df_1 = df_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
# df_1 = df_1.drop(columns='run_date')

# print("Total sample size of the Infinity batch after merging runid is " + str(df_1.shape[0]))
# df_1.head()


Total sample size of the Infinity batch after merging runid is 1


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
0,C4221015,13431002_Cycle 1 Day 1,A1218829,B00888193,13431002,Cycle 1 Day 1,1,SUCCESS,NaN,SNV,NaN,PIN1,19,9958804.0,3.0,A124S,G>T,c.370G>T,NM_006221.4,51.12,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs138056875,NaN,NaN,NaN,NaN,5935.0,5.85,NaN,0.060326,Detected,0.056315,22.78,High,Not Detected,22.85,3.5,0.0,11/16/2024,06/20/2024,01/17/2025,CRC,Not Applicable,Not Applicable,250106_A00277_1208_BHWTHFDSXC


In [8]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_2, df_no_found = add_fc_dir_column(df_1)

print("Total sample size of the Infinity batch with fc_dir is " + str(df_2.shape[0]) + "\n") 
print("Total sample size of the Infinity batch with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_2.head()



Total sample size of the Infinity batch with fc_dir is 42

Total sample size of the Infinity batch with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir
21,XTX101-0102-001,NaN,A1210253,A121025301,104-017,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.346847,42.92,4.5,NaN,11/14/24,11/13/24,11/25/24,241121_A01810_0520_BHT73GDSXC,/ghsfa/ivd/flowcentral/241121_A01810_0520_BHT73GDSXC/sirius
13,XTX101-0102-001,NaN,A1226429,A122642901,146-004,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Detected,Detected,0.119741,37.01,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius
18,XTX101-0102-001,NaN,A1226435,A122643501,146-003,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.057710,77.38,NaN,NaN,11/30/24,11/29/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius
0,XTX101-0102-001,NaN,A1226443,A122644301,140-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.003628,58.33,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius
28,XTX101-0102-001,NaN,A1226445,A122644501,140-005,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.012115,49.20,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius


## Generate S3 MB CRC caller outputs (Sirius 1.5.0)
###### Git clone Sirius-1.5.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.5.0-RLS/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L610-L623
###### Cloned Sirius-1.5.0_Release_Branch dir: /home/hazhang/projects/Repo/bip-products/

In [6]:
# Make sure update the epicaller to 2.0.3 or above version
import epicaller
print(epicaller.__version__)

2.0.3


In [18]:
import epicaller
import epicaller.methylation.methyl_caller

def mb_crc_caller_sirius150_wrapper_slurm(sample_meta_df, workdir):
    region_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.classification_space.tsv'
    control_region_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.positive_control_regions.tsv'
    mr_model_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.threshold_set_to_zero.tsv'

    jobs_dir = f"{workdir}/MB_CRC_log"
    output_dir = f"{workdir}/MB_CRC_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]
        flowcell_id = row["runid"]
        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        
        # Check if the molecule file exists
        if not os.path.exists(mbd_hyper_molecules):
            continue

        job_file = f"{jobs_dir}/MB_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=30gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=00:10:00\n"
                           f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n\n")

        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write("python3 -m epicaller.methylation.methyl_caller " + 
                             f"--hyper-molecule-file {mbd_hyper_molecules} " + 
                             f"--region-file {region_file} " +
                             f"--control-region-file {control_region_file} " + 
                             f"--model-file {mr_model_file} " +
                             "--model-name crc --min-frag-len 120 --max-frag-len 240 --min-cg-count 12 --max-cg-count 30 --cutsite-count-cutoff 1 --min-mapq 60 " + 
                             f"--sample-id {run_sample_id} --flowcell-id {flowcell_id} --outdir {output_dir}/")
            # print(job_output)    
        os.system(f"sbatch {job_file}")

In [19]:
# Generate the MB S3 CRC caller outputs (Sirius 1.5.0)
mb_crc_caller_sirius150_wrapper_slurm(df_2, f"{work_dir}")

Submitted batch job 97846581
Submitted batch job 97846582
Submitted batch job 97846583
Submitted batch job 97846584
Submitted batch job 97846586
Submitted batch job 97846587
Submitted batch job 97846589
Submitted batch job 97846591
Submitted batch job 97846593
Submitted batch job 97846594
Submitted batch job 97846596
Submitted batch job 97846597
Submitted batch job 97846599
Submitted batch job 97846601
Submitted batch job 97846602
Submitted batch job 97846604
Submitted batch job 97846606
Submitted batch job 97846608
Submitted batch job 97846610
Submitted batch job 97846612
Submitted batch job 97846614
Submitted batch job 97846615
Submitted batch job 97846617
Submitted batch job 97846619
Submitted batch job 97846621
Submitted batch job 97846623
Submitted batch job 97846624
Submitted batch job 97846626
Submitted batch job 97846627
Submitted batch job 97846629
Submitted batch job 97846631
Submitted batch job 97846633
Submitted batch job 97846635
Submitted batch job 97846636
Submitted batc

In [ ]:
# # Retrieve MB CRC score and CRC call of Sirius 1.5.0 result
# df_3 = df_2.copy()

# # Prep some columns for comparison
# df_3["MB_CRC_call_sirius150_version"] = np.nan
# df_3["MB_CRC_epi_score_sirius150_version"] = np.nan

# # # Optional for pulling current BIP run version (only works BIP prior to Sirius 1.6.x; adding /sirius subfolder for Sirius 1.6.x)
# # df_3["MB_CRC_call_BIP_run_version"] = np.nan
# # df_3["MB_CRC_BIP_score_BIP_run_version"] = np.nan

# mb_caller_result_folder = f"{work_dir}/MB_CRC_output"


# for index, crc_row in df_3.iterrows():
#     run_sample_id = crc_row["GHSampleID"]
#     msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}.msre_crc_call.hdr.tsv" 
#     msre_df = pd.read_table(msre_mr_file)

#     # Retrieve the Sirius 1.5.0 results
#     df_3.at[index, "MB_CRC_epi_score_sirius150_version"] = msre_df.loc[0, "methyl_score"]
#     df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected" if msre_df.loc[0, "methyl_call"] == 1 else "Not Detected"

#     # # Optional step
#     # # Retrieve the current BIP run results
#     # fc_dir = crc_row["fc_dir"]
#     # msre_mr_BIP_file = f"{fc_dir}/{run_sample_id}/{run_sample_id}.msre_all_call.hdr.tsv" 
#     # msre_BIP_df = pd.read_table(msre_mr_BIP_file)

#     # msre_BIP_df = msre_BIP_df[msre_BIP_df['model_name'] == 'CRC'].reset_index(drop=True)
#     # if not msre_BIP_df.empty:
#     #     df_3.at[index, "MB_CRC_BIP_score"] = msre_BIP_df.loc[0, "methyl_score"]
#     #     df_3.at[index, "MB_CRC_BIP_call"] = "Detected" if msre_BIP_df.loc[0, "methyl_call"] == 1 else "Not Detected"

# df_3.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_sirius150_version,MB_CRC_epi_score_sirius150_version
21,XTX101-0102-001,NaN,A1210253,A121025301,104-017,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.346847,42.92,4.5,NaN,11/14/24,11/13/24,11/25/24,241121_A01810_0520_BHT73GDSXC,/ghsfa/ivd/flowcentral/241121_A01810_0520_BHT73GDSXC/sirius,Detected,49.155813
13,XTX101-0102-001,NaN,A1226429,A122642901,146-004,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Detected,Detected,0.119741,37.01,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,37.811887
18,XTX101-0102-001,NaN,A1226435,A122643501,146-003,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.057710,77.38,NaN,NaN,11/30/24,11/29/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,41.845025
0,XTX101-0102-001,NaN,A1226443,A122644301,140-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.003628,58.33,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,17.901997
28,XTX101-0102-001,NaN,A1226445,A122644501,140-005,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.012115,49.20,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,26.892025


In [14]:
# Retrieve MB CRC score and CRC call of Sirius 1.5.0 result (IMPORTANT NOTES: only work for late-stage CRC samples and run with Sirius 1.6.1)
df_3 = df_2.copy()

# Prep some columns for comparison
df_3["MB_CRC_call_sirius150_version"] = np.nan
df_3["MB_CRC_epi_score_sirius150_version"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_CRC_output"

for index, crc_row in df_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]
    msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}.msre_crc_call.hdr.tsv" 
    msre_df = pd.read_table(msre_mr_file)

    # Retrieve and recalculate the methylation score (subtracting 20.2 as epicaller used -20.2 (98% spec) as cutoff)
    original_score = msre_df.loc[0, "methyl_score"]
    crc_methyl_score = original_score - 20.2
    df_3.at[index, "MB_CRC_epi_score_sirius150_version"] = crc_methyl_score

    # Retrieve tumor methylation score
    tf_score = crc_row["Tumor_methylation_score"]

    # Apply the detection rules of Sirius 1.6.1
    if crc_methyl_score >= -21.36903: 
        df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
    elif (crc_methyl_score >= -22.50785) and (tf_score >= 0.0005):
        df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
    else:
        df_3.at[index, "MB_CRC_call_sirius150_version"] = "Not Detected"


df_3.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_sirius150_version,MB_CRC_epi_score_sirius150_version
21,XTX101-0102-001,NaN,A1210253,A121025301,104-017,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.346847,42.92,4.5,NaN,11/14/24,11/13/24,11/25/24,241121_A01810_0520_BHT73GDSXC,/ghsfa/ivd/flowcentral/241121_A01810_0520_BHT73GDSXC/sirius,Detected,28.955813
13,XTX101-0102-001,NaN,A1226429,A122642901,146-004,Colorectal adenocarcinoma,Stage III,NaN,SUCCESS,NaN,Detected,Detected,0.119741,37.01,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,17.611887
18,XTX101-0102-001,NaN,A1226435,A122643501,146-003,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.057710,77.38,NaN,NaN,11/30/24,11/29/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,21.645025
0,XTX101-0102-001,NaN,A1226443,A122644301,140-001,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.003628,58.33,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,-2.298003
28,XTX101-0102-001,NaN,A1226445,A122644501,140-005,Colorectal adenocarcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.012115,49.20,NaN,NaN,11/30/24,11/27/24,12/13/24,241205_A00277_1180_AHTGM7DSXC,/ghsfa/ivd/flowcentral/241205_A00277_1180_AHTGM7DSXC/sirius,Detected,6.692025


In [ ]:
# df_3 = df_2.copy()

# # Prep some columns for comparison
# df_3["MB_CRC_call_sirius150_version"] = np.nan
# df_3["MB_CRC_epi_score_sirius150_version"] = np.nan

# mb_caller_result_folder = f"{work_dir}/MB_CRC_output"

# for index, crc_row in df_3.iterrows():
#     run_sample_id = crc_row["GHSampleID"]
#     msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}.msre_crc_call.hdr.tsv" 
#     msre_df = pd.read_table(msre_mr_file)

#     # Retrieve and recalculate the methylation score (subtracting 20.2)
#     original_score = msre_df.loc[0, "methyl_score"]
#     crc_methyl_score = original_score - 20.2
#     df_3.at[index, "MB_CRC_epi_score_sirius150_version"] = crc_methyl_score

#     # Retrieve tumor methylation score
#     tf_score = crc_row["Tumor_methylation_score"]

#     # Retrieve the BIP version and cancer stage from the dataframe (two columns are fake columns)
#     BIP_version = crc_row["BIP_version"]
#     Cancer_stage = crc_row["Cancer_stage"]

#     # Apply the detection rules based on BIP version and cancer stage
#     if BIP_version == "Sirius 1.6.1":
#         if Cancer_stage == "late-stage":
#             if crc_methyl_score >= -21.36903:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
#             elif (crc_methyl_score >= -22.50785) and (tf_score >= 0.0005):
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
#             else:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Not Detected"
#         elif Cancer_stage == "early-stage":
#             if crc_methyl_score >= -20.31064:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
#             else:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Not Detected"
#         else:
#             df_3.at[index, "MB_CRC_call_sirius150_version"] = np.nan

#     elif BIP_version == "Sirius 1.6.0":
#         if Cancer_stage == "late-stage":
#             if crc_methyl_score >= -21.249985:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
#             elif (crc_methyl_score >= -22.4522) and (tf_score >= 0.0005):
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
#             else:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Not Detected"
#         elif Cancer_stage == "early-stage":
#             if crc_methyl_score >= -20.2:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Detected"
#             else:
#                 df_3.at[index, "MB_CRC_call_sirius150_version"] = "Not Detected"
#         else:
#             df_3.at[index, "MB_CRC_call_sirius150_version"] = np.nan
#     else:
#         df_3.at[index, "MB_CRC_call_sirius150_version"] = np.nan

# df_3.head()


In [ ]:
df_flipped_call = df_3[df_3['MB_CRC_call_sirius150_version'] != df_3['Tumor_methylation_status']]
print("Total CRC samples with flipped call is " + str(df_flipped_call.shape[0]) + "\n")

# Save the data
# df_flipped_call.to_csv(f"{work_dir}/20250121_Pfizer_SOW51core_Batch3_18Samples_InfinityReport_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20241220_GuardantHealthIST_IST01_Batch1_405Samples_InfinityReport_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20250120_Pfizer_SOW46Amendment1TP3_Batch9_20Samples_G360Report_flipped_calls.csv", index = False)
# df_flipped_call.to_csv(f"{work_dir}/20250120_Pfizer_SOW46Amendment1TP5_Batch7_33Samples_InfinityReport_flipped_calls.csv", index = False)
df_flipped_call.to_csv(f"{work_dir}/XTX_monitoring_Sirius161_by01242025_flipped_calls.csv", index = False)

df_flipped_call



Total CRC samples with flipped call is 1



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Cancertype,Tumor_stage,Visit_name,Sample_status,Sample_comment,ctDNA_detected,Tumor_methylation_status,Tumor_methylation_score,cfDNA_ng_yield,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,runid,fc_dir,MB_CRC_call_sirius150_version,MB_CRC_epi_score_sirius150_version
20,XTX101-0102-001,NaN,A1243649,A124364901,101-027,Cecum carcinoma,"Stage IIIB/IV NSCLC, Stage III/IV - Other Cancer Type",NaN,SUCCESS,NaN,Detected,Detected,0.000143,21.39,NaN,NaN,12/18/24,12/17/24,1/2/25,241221_A01020_0945_AHWTCMDSXC,/ghsfa/ivd/flowcentral/241221_A01020_0945_AHWTCMDSXC/sirius,Not Detected,-21.906225
